### Linear Regression 3D

Implementation of linear regression with two features

***
#### Environment
`conda activate sklearn-env`

***
#### Goals

- Mathematical intuition
- Run gradient descent
- Visualize cost values during gradient descent run
- Visualize (3D) hypothetis during gradient descent run
- Predict (score) MPG for the first dataset row  

***
$\mathbf{\text{Multivariate Linear regression - dual features}}$<br>
***
- #### Hypothesis $$h_{\theta}(x) = {\theta}_{0} +{\theta}_{1}X_{1}+{\theta}_{2}X_{2}$$ <br> or <br> $$h_{\theta}(x) = {\theta}^TX$$

- #### Cost function J $$ J(\theta) = \frac{1}{2m} \sum _{i=1} ^{m} (h_{\theta}(x^{(i)}) - y^{(i)})^2$$

- #### Gradient descent <br> 
  repeat until convergence $${\theta}_{j} := {\theta}_{j} - \alpha \frac{\partial}{\partial \theta_{j}} J_{\theta}$$ 
   <br> or <br>
  repeat until convergence { $${\theta}_{0} := {\theta}_{0} - \alpha \frac{1}{m} \sum _{i=1} ^{m} (h_{\theta}(x^{(i)}) - y^{(i)}) $$  <br> $${\theta}_{1} := {\theta}_{1} - \alpha \frac{1}{m} \sum _{i=1} ^{m} (h_{\theta}(x^{(i)}) - y^{(i)})  x^{(i)}_1 $$ <br> $${\theta}_{2} := {\theta}_{2} - \alpha \frac{1}{m} \sum _{i=1} ^{m} (h_{\theta}(x^{(i)}) - y^{(i)})  x^{(i)}_2 $$  }

- #### Features scaling $$ x_{i} := \frac{x_{i} - \mu_{i}}{s_{i}}$$

***

#### Basic python imports for panda (dataframe), numpy (numeric computation), matplotlib (visualization) packages

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


# Make numpy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)

#### Dataset load from CSV located on UCI website.

http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data  
If the URL does not work the dataset can be loaded from the data folder `./data/auto-mpg.data`.

In [ ]:
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'
column_names = ['MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight',
                'Acceleration', 'Model Year', 'Origin']

raw_dataset = pd.read_csv(url, names=column_names,
                          na_values='?', comment='\t',
                          sep=' ', skipinitialspace=True)

#### Keep original dataset imutabe and copy its content in a new dataset for further changes

In [ ]:
dataset = raw_dataset.copy()
dataset.tail()

#### Display total count of missing values 

In [ ]:
dataset.isna().sum()

#### Eliminate missing values from dataset

In [ ]:
dataset = dataset.dropna()

#### Use only `MPG` `Weight` and `Horsepower` columns

In [ ]:
train_dataset = dataset.drop(['Cylinders', 'Displacement', 'Acceleration', 'Model Year', 'Origin'], axis=1, inplace=False)
train_dataset.tail()

#### Separate features(columns used to predict) and labels (predicted) columns

In [ ]:
train_features = train_dataset.copy()
train_labels = train_features.pop('MPG')

#### Rescale features column to have values in [0,1] interval

In [ ]:
stats = train_features.describe().transpose()[['mean', 'std', 'count', 'min', 'max']]
normalized_train_features = (train_features - stats['mean'].transpose()) /  stats['std'].transpose()
normalized_train_features.tail()

#### Insert $X_0$ column as static value (1.0) in order to match $\theta$ vector length

In [ ]:
normalized_ones_features = normalized_train_features.copy()
normalized_ones_features.insert(0, 'Oness', 1.0)
normalized_ones_features.head()

#### Run gradient descent and compute cost at every step 

In [ ]:
def costFunction(X, y , theta):
    m = len(y)
    sqHipe = np.matmul(X , theta) - y
    cost = (1/(2*m)) * np.sum(sqHipe * sqHipe)
    return cost
    

def gradientDescent(X, y, theta, alpha, num_iter):
    m = len(y)
    jurnal = np.zeros(num_iter)
    theta_jurnal = np.zeros((num_iter, len(theta)))
    for iter in range(num_iter):
        theta = theta - alpha * (1/m) * np.sum(((np.matmul(X , theta) - y).transpose() * X.transpose()).transpose(), axis=0)
        jurnal[iter] = costFunction(X, y, theta)
        theta_jurnal[iter] = theta
    return theta, jurnal, theta_jurnal


theta = np.zeros(len(normalized_ones_features.columns))
alpha = 0.01;
num_iters = 400;
theta , jurnal, theta_jurnal = gradientDescent(normalized_ones_features.to_numpy(), train_labels.to_numpy(), theta, alpha, num_iters);
theta
print(f"Hypothesis: h(X)= {theta[0]} + {theta[1]} * X1 +  {theta[2]} * X2 ")


#### #### Visualize cost reduction

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

figure(figsize=(10, 10), dpi=80)

plt.plot(range(num_iters), jurnal)
plt.xlabel('Number of iterations' , fontsize=15)
plt.ylabel('Cost J' , fontsize=15)
plt.show()

#### Visualize (3D) hypothesis plan during gradient descent run 

In [ ]:
X = np.linspace(stats['min']['Weight'],stats['max']['Weight'],  2)
X_scaled = (X-stats['mean']['Weight']) / stats['std']['Weight']

Y = np.linspace(stats['min']['Horsepower'],stats['max']['Horsepower'],  2)
Y_scaled = (Y-stats['mean']['Horsepower']) / stats['std']['Horsepower']

XY=np.ones((len(X_scaled),len(theta)))
XY[:,1:]=X_scaled[:,np.newaxis]
XY[:,2:]=Y_scaled[:,np.newaxis]

A, B = np.meshgrid(X, Y)

z = np.matmul(XY , theta)

A, B = np.meshgrid(X, Y)

Z = np.ones((2,2))
Z_jurnal = np.ones((2,2))

Z[0,0]= np.matmul(np.array([1.0, X_scaled[0], Y_scaled[0]]), theta)
Z[0,1]= np.matmul(np.array([1.0, X_scaled[0], Y_scaled[1]]), theta)
Z[1,0]= np.matmul(np.array([1.0, X_scaled[1], Y_scaled[0]]), theta)
Z[1,1]= np.matmul(np.array([1.0, X_scaled[1], Y_scaled[1]]), theta)


In [ ]:
class Container:
    def __init__(self, surface):
        self.surface = surface

    def update_surface(self, other):
        self.surface = other


In [ ]:
import matplotlib.animation

fig = plt.figure(figsize=(10, 10), dpi=80)
ax = fig.add_subplot(111, projection='3d')
ax.set_title('Theta variance')

ax.set_xlabel(r'Weight', fontsize=15)
ax.set_ylabel(r'Horsepower', fontsize=15)
ax.set_zlabel(r'MPG', fontsize=15)

ax.scatter(train_features['Weight'], train_features['Horsepower'] ,train_labels)

#ax.legend(loc="upper left",  shadow=True, title="Legend")
#ax.get_legend().get_title().set_color("red")

ax = fig.gca()

surf = ax.plot_surface(A, B, Z, color = 'b' #, cmap=cm.coolwarm
                       ,linewidth=0, antialiased=False, alpha=0.4, label="Theta "+ str(theta))

surf_other = ax.plot_surface(A, B, Z_jurnal, color = 'r' #, cmap=cm.coolwarm
                       ,linewidth=0, antialiased=False, alpha=0.3, label="Theta []")
container = Container(surf_other)

def animate(i):
    Z_jurnal[0,0]= np.matmul(np.array([1.0, X_scaled[0], Y_scaled[0]]), theta_jurnal[i*8])
    Z_jurnal[0,1]= np.matmul(np.array([1.0, X_scaled[0], Y_scaled[1]]), theta_jurnal[i*8])
    Z_jurnal[1,0]= np.matmul(np.array([1.0, X_scaled[1], Y_scaled[0]]), theta_jurnal[i*8])
    Z_jurnal[1,1]= np.matmul(np.array([1.0, X_scaled[1], Y_scaled[1]]), theta_jurnal[i*8])

    #surf_other.set_data(A, B,  Z_jurnal)
    #ax.clear
    container.surface.remove()
    container.update_surface( ax.plot_surface(A, B, Z_jurnal, color = 'r' #, cmap=cm.coolwarm
                                 ,linewidth=0, antialiased=False, alpha=0.2, label="Theta: "+ str(theta_jurnal[i*8])))

ani = matplotlib.animation.FuncAnimation(fig, animate, frames=int(num_iters/8))

from IPython.display import HTML
anim = HTML(ani.to_jshtml())
anim

In [ ]:
score_input = (train_features.head(1).to_numpy()[0] -  stats['mean'].transpose()) /  stats['std'].transpose();
score_elem = np.insert(score_input.to_numpy(),0,1,axis=0)

In [ ]:
test_mpg = np.matmul(score_elem , theta)
print("Predicted MPG:" ,test_mpg,  " actual value ", train_labels.head(1).to_numpy()[0])